<a href="https://colab.research.google.com/github/sajkcaz19/Optimizador-de-itinerarios-de-viajes-con-fines-gastron-micos-en-el-Centro-Hist-rico-de-Cartagena/blob/main/INTERFAZ_CON_ONTOLOG%C3%8DA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit pandas folium streamlit-folium pyngrok owlready2 rdflib -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 47.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 88.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.6/524.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.4/565.4 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 114.2 MB/s eta 0:00:00


In [ ]:
from google.colab import files

uploaded = files.upload()
ontology_path = list(uploaded.keys())[0]
print(f"✅ Ontología cargada: {ontology_path}")


Saving Gastronomy_Ontology.ttl to Gastronomy_Ontology.ttl
✅ Ontología cargada: Gastronomy_Ontology.ttl


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import random
import folium
from streamlit_folium import st_folium
from owlready2 import get_ontology

# ===============================
# CONFIGURACIÓN
# ===============================
st.set_page_config(page_title="Agente Gastronómico Inteligente", page_icon="🍽️", layout="wide")
st.title("🍽️ Agente Inteligente Gastronómico – Centro Histórico de Cartagena")

# ===============================
# CARGA DE ONTOLOGÍA
# ===============================
from rdflib import Graph
from pathlib import Path

try:
    ontology_path = "/content/Gastronomy_Ontology.ttl"
    owl_path = "/content/Gastronomy_Ontology.owl"

    # Si existe archivo .ttl, convertir a OWL antes de cargar
    if Path(ontology_path).exists():
        st.sidebar.info("📄 Convirtiendo ontología de TTL a OWL...")
        g = Graph()
        g.parse(ontology_path, format="turtle")
        g.serialize(destination=owl_path, format="xml")
        st.sidebar.success("✅ Conversión exitosa. Cargando ontología...")

        from owlready2 import get_ontology
        onto = get_ontology(f"file://{owl_path}").load()
        st.sidebar.success("✅ Ontología cargada correctamente desde archivo convertido.")
    else:
        st.sidebar.error("❌ No se encontró el archivo de ontología.")
        onto = None

except Exception as e:
    st.sidebar.error(f"❌ Error al procesar la ontología: {e}")
    onto = None


# ===============================
# INTERFAZ DEL USUARIO
# ===============================
st.sidebar.header("👁️ Entrada del Usuario")
nombre = st.sidebar.text_input("Nombre", "Oriana")
preferencia = st.sidebar.selectbox("Tipo de comida", ["vegano", "caribeña", "café", "postres"])
presupuesto = st.sidebar.slider("Presupuesto (COP)", 20000, 100000, 60000, step=5000)
tiempo_disp = st.sidebar.slider("Tiempo disponible (horas)", 1, 6, 3)
st.sidebar.markdown("---")
st.sidebar.subheader("Condiciones del entorno 🌦️")
clima = st.sidebar.selectbox("Clima actual", ["Soleado", "Lluvioso"])
evento = st.sidebar.checkbox("🚫 Un restaurante está cerrado")

# ===============================
# BASE DE DATOS LOCAL
# ===============================
restaurantes = pd.DataFrame([
    {"nombre":"Verde Vida","tipo":"vegano","precio":25000,"distancia_km":0.8,"al_aire_libre":True,"lat":10.4235,"lon":-75.5493,"tiempo_visita_h":1.0},
    {"nombre":"Café San Alberto","tipo":"café","precio":15000,"distancia_km":0.3,"al_aire_libre":False,"lat":10.4251,"lon":-75.5487,"tiempo_visita_h":0.5},
    {"nombre":"La Palettería","tipo":"postres","precio":10000,"distancia_km":0.5,"al_aire_libre":True,"lat":10.4258,"lon":-75.5501,"tiempo_visita_h":0.5},
    {"nombre":"La Cocina de Pepina","tipo":"caribeña","precio":40000,"distancia_km":1.2,"al_aire_libre":False,"lat":10.4243,"lon":-75.5507,"tiempo_visita_h":1.2},
    {"nombre":"Quero Arepa","tipo":"vegano","precio":20000,"distancia_km":0.6,"al_aire_libre":True,"lat":10.4260,"lon":-75.5482,"tiempo_visita_h":0.8}
])

# ===============================
# FUNCIÓN DE RAZONAMIENTO ONTOLÓGICO
# ===============================
def es_compatible(preferencia_usuario, tipo_restaurante):
    """Evalúa compatibilidad entre preferencia y restaurante según la ontología."""
    try:
        if onto is None:
            return preferencia_usuario.lower() == tipo_restaurante.lower()

        clase_usuario = onto[preferencia_usuario.capitalize()]
        clase_rest = onto[tipo_restaurante.capitalize()]
        compatibles = [rel.name for rel in clase_usuario.compatibleCon]
        if tipo_restaurante.capitalize() in compatibles:
            return True
    except:
        pass
    return preferencia_usuario.lower() == tipo_restaurante.lower()

# ===============================
# ESTADO DE SESIÓN
# ===============================
if "itinerario" not in st.session_state:
    st.session_state["itinerario"] = None
    st.session_state["clima"] = None
    st.session_state["evento"] = None

# ===============================
# BOTÓN DE GENERACIÓN
# ===============================
generar = st.sidebar.button("Generar Itinerario")

if generar:
    st.session_state["clima"] = clima
    st.session_state["evento"] = evento
    df = restaurantes.copy()

    # Filtrado por clima
    if clima == "Lluvioso":
        df = df[df["al_aire_libre"] == False]
        st.warning("🌧️ Clima lluvioso: se excluyen restaurantes al aire libre.")

    # Evento aleatorio de cierre
    if evento and not df.empty:
        cerrado = random.choice(df["nombre"].tolist())
        df = df[df["nombre"] != cerrado]
        st.warning(f"🚫 El restaurante **{cerrado}** está cerrado temporalmente.")

    # Compatibilidad semántica
    filtro = df[df["tipo"].apply(lambda t: es_compatible(preferencia, t))]

    if len(filtro) == 0:
        st.error("No hay restaurantes compatibles según la ontología.")
        st.session_state["itinerario"] = None
    else:
        filtro["utilidad"] = filtro.apply(lambda r: (1 / (r["distancia_km"] + 0.1)) + (1 / (r["precio"] / 10000)), axis=1)
        itinerario = filtro.sort_values(by="utilidad", ascending=False)
        st.session_state["itinerario"] = itinerario
        st.success(f"✅ Se generó un nuevo itinerario con {len(itinerario)} restaurantes.")

        # Mostrar compatibilidades según la ontología
        try:
            clase_usuario = onto[preferencia.capitalize()]
            compatibles = [rel.name for rel in clase_usuario.compatibleCon]
            if compatibles:
                st.info(f"🧩 Según la ontología, '{preferencia}' es compatible con: {', '.join(compatibles)}")
        except:
            pass

# ===============================
# MOSTRAR RESULTADOS
# ===============================
if st.session_state["itinerario"] is not None:
    itinerario = st.session_state["itinerario"]
    st.header("🧭 Itinerario Gastronómico Actual")
    st.dataframe(itinerario[["nombre", "tipo", "precio", "distancia_km", "utilidad"]])

    tiempo_total = itinerario["tiempo_visita_h"].sum()
    costo_total = itinerario["precio"].sum()
    col1, col2 = st.columns(2)
    col1.metric("🕒 Tiempo total estimado", f"{tiempo_total:.2f} horas")
    col2.metric("💰 Costo total estimado", f"{costo_total:,.0f} COP")

    # Mapa
    st.header("🗺️ Visualización en Mapa")
    m = folium.Map(location=[10.424, -75.549], zoom_start=16)
    for i, r in enumerate(itinerario.itertuples(), 1):
        folium.Marker([r.lat, r.lon],
                      popup=f"{i}. {r.nombre}<br>{r.tipo}<br>💰{r.precio} COP",
                      icon=folium.Icon(color="green", icon="cutlery", prefix="fa")).add_to(m)
    st_folium(m, width=700, height=450)

    # Adaptación dinámica
    st.header("🔁 Adaptación Dinámica del Agente")
    if st.button("Simular Cambio de Clima 🌧️"):
        st.warning("El clima cambió a lluvioso, recalculando itinerario…")
        nuevo = itinerario[itinerario["al_aire_libre"] == False]
        if len(nuevo) > 0:
            st.session_state["itinerario"] = nuevo
            st.success(f"Itinerario actualizado ({len(nuevo)} restaurantes disponibles).")
            for i, r in enumerate(nuevo.itertuples(), 1):
                st.write(f"**{i}. {r.nombre}** ({r.tipo})")
        else:
            st.error("Ningún restaurante disponible bajo condiciones lluviosas.")


Writing app.py


In [ ]:
from pyngrok import ngrok
import subprocess, time

# Reemplaza con tu token personal de ngrok
NGROK_AUTH_TOKEN = "33u3HXnySJkUcWYApyTXNbjGa17_2zaDtHKpGNQjtDjoRq7uv"

ngrok.set_auth_token(NGROK_AUTH_TOKEN)
ngrok.kill()
public_url = ngrok.connect(8501).public_url
print(f"🌍 Tu aplicación está disponible en:\n👉 {public_url}")
time.sleep(2)
subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501"])


🌍 Tu aplicación está disponible en:
👉 https://prebronchial-elvin-corporate.ngrok-free.dev


<Popen: returncode: None args: ['streamlit', 'run', 'app.py', '--server.port...>